# Logistic regression

In [1]:
from pathlib import Path
data_path = Path('/home/juvid/Python-and-Spark-for-Big-Data-master/Course_Notes/Spark_for_Machine_Learning/Logistic_Regression/sample_libsvm_data.txt')

In [2]:
from pyspark.ml.classification import LogisticRegression

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('lg').getOrCreate()

21/08/13 09:00:41 WARN Utils: Your hostname, GBLON1WLZ13699 resolves to a loopback address: 127.0.1.1; using 10.164.76.148 instead (on interface eth2)
21/08/13 09:00:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/08/13 09:00:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
df = spark.read.format('libsvm').load(str(data_path))

21/08/13 09:00:54 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [6]:
df.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [7]:
log_reg_model = LogisticRegression()

In [8]:
model = log_reg_model.fit(df)

21/08/13 09:01:04 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/08/13 09:01:04 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [9]:
summary = model.summary
summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [10]:
summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111051...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126979...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170301...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200602...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990277...|[0.99999999760673...|       0.0|
|  1.0|(692,[158,159,160...|[-20.337256674833...|[1.47109814695578...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.595579753418...|[3.08850168102648...|       1.0|
|  0.0|(692,[154,155,156...|[19.2708803215613...|[0.99999999572670...|       0.0|
|  0.0|(692,[127

In [11]:
train_data, test_data = df.randomSplit([0.7,0.3])

In [12]:
fit = log_reg_model.fit(train_data)

In [13]:
predictions = fit.evaluate(test_data)

In [14]:
predictions.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[98,99,100,1...|[22.3349609001251...|[0.99999999980045...|       0.0|
|  0.0|(692,[100,101,102...|[9.79780917009642...|[0.99994442987058...|       0.0|
|  0.0|(692,[122,123,148...|[22.2922011501123...|[0.99999999979173...|       0.0|
|  0.0|(692,[124,125,126...|[27.0966140909980...|[0.99999999999829...|       0.0|
|  0.0|(692,[124,125,126...|[21.9111753228474...|[0.99999999969514...|       0.0|
|  0.0|(692,[124,125,126...|[21.5533752312619...|[0.99999999956399...|       0.0|
|  0.0|(692,[126,127,128...|[31.3753676275591...|[0.99999999999997...|       0.0|
|  0.0|(692,[127,128,129...|[22.0880633843459...|[0.99999999974456...|       0.0|
|  0.0|(692,[127,128,129...|[22.9704703677922...|[0.99999999989430...|       0.0|
|  0.0|(692,[152

## Using evaluators

In [15]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [16]:
my_eval = BinaryClassificationEvaluator()

In [17]:
roc = my_eval.evaluate(predictions.predictions)
roc

1.0

In [18]:
my_eval

BinaryClassificationEvaluator_32971474bfef

# Using pipelines on the Titanic dataset

In [21]:
data_dir = Path('/home/juvid/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Logistic_Regression/titanic.csv')

In [22]:
df = spark.read.csv(str(data_dir), header=True, inferSchema=True)

In [24]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [26]:
cols_of_interest = ['Survived', 'Pclass','Sex',
 'Age',
 'SibSp',
 'Parch','Fare','Embarked']

In [27]:
df_filtered = df.select(cols_of_interest)

Let's drop missing data

In [29]:
df_clean = df_filtered.na.drop()

In [30]:
df_clean.show()

+--------+------+------+----+-----+-----+-------+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|
|       1|     1|female|38.0|    1|    0|71.2833|       C|
|       1|     3|female|26.0|    0|    0|  7.925|       S|
|       1|     1|female|35.0|    1|    0|   53.1|       S|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|
|       0|     1|  male|54.0|    0|    0|51.8625|       S|
|       0|     3|  male| 2.0|    3|    1| 21.075|       S|
|       1|     3|female|27.0|    0|    2|11.1333|       S|
|       1|     2|female|14.0|    1|    0|30.0708|       C|
|       1|     3|female| 4.0|    1|    1|   16.7|       S|
|       1|     1|female|58.0|    0|    0|  26.55|       S|
|       0|     3|  male|20.0|    0|    0|   8.05|       S|
|       0|     3|  male|39.0|    1|    5| 31.275|       S|
|       0|     3|female|14.0|    0|    0| 7.8542|       

## Working with categorical variables...

1. Create a string indexer (convert string into integer)
2. One-hot encode

In [31]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer

In [33]:
gender_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')

In [34]:
embark_indexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex', outputCol='EmbarkVec')

In [35]:
assembler = VectorAssembler(inputCols=['Pclass', 'SexVec', 'EmbarkVec', 'Age', 'SibSp', 'Parch', 'Fare'],
                            outputCol='features')

In [36]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [37]:
log_reg_titanic = LogisticRegression(featuresCol='features', labelCol='Survived')

In [38]:
pipeline = Pipeline(stages=[gender_indexer, embark_indexer, 
                            gender_encoder, embark_encoder, 
                            assembler, 
                            log_reg_titanic])

In [45]:
train_data, test_data = df_clean.randomSplit([0.7, 0.3])

In [46]:
model = pipeline.fit(train_data)

In [47]:
results = model.transform(test_data)

In [50]:
results.printSchema()

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- SexIndex: double (nullable = false)
 |-- EmbarkIndex: double (nullable = false)
 |-- SexVec: vector (nullable = true)
 |-- EmbarkVec: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [48]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [52]:
evaluation = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

In [56]:
results_filter = results.select(['Survived', 'prediction', 'rawPrediction', 'probability'])
results_filter.show()

+--------+----------+--------------------+--------------------+
|Survived|prediction|       rawPrediction|         probability|
+--------+----------+--------------------+--------------------+
|       0|       1.0|[-3.4943226322872...|[0.02947420142437...|
|       0|       1.0|[-1.8085479464474...|[0.14081371112918...|
|       0|       1.0|[-0.4652934083648...|[0.38573083396425...|
|       0|       1.0|[-2.0336845182735...|[0.11571137970374...|
|       0|       1.0|[-2.2692325041424...|[0.09370337000422...|
|       0|       1.0|[-3.3685229291996...|[0.03329381610415...|
|       0|       1.0|[-2.7822880357179...|[0.05828883540416...|
|       0|       1.0|[-1.1358209823585...|[0.24308845967279...|
|       0|       1.0|[-0.1457798947540...|[0.46361943281299...|
|       0|       1.0|[-0.2408112754333...|[0.44008643435118...|
|       0|       1.0|[-0.2991691752241...|[0.42576059776104...|
|       0|       1.0|[-0.0031534820034...|[0.49921163015245...|
|       0|       1.0|[-0.1148721538936..

In [60]:
result_row = results_filter.head(1)[0]
result_row

Row(Survived=0, prediction=1.0, rawPrediction=DenseVector([-3.4943, 3.4943]), probability=DenseVector([0.0295, 0.9705]))

In [63]:
result_row['rawPrediction'][0]

-3.4943226322872714

In [70]:
from scipy.special import expit
import numpy as np
np.isclose(expit(result_row['rawPrediction'][0]), result_row['probability'][0])

True

In [71]:
evaluation.evaluate(results)  # ROC

0.7704711514235323